In [16]:
#!/hetghome/hetgsoft/anaconda3/bin/python3.9
#old saj version
#sed -i 's/\r//g' cal_insta_map.py
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import math as m
import math
from scipy.optimize import fsolve
from scipy import linalg
import csv
import pandas as pd
import sys
import numpy as np
import os
from scipy.integrate import RK45
import time

#######################################
#Lmu = 200
LLmu = [300]
AR = [0.7]
theta = m.pi/2
alpha=5/3
ar = 0.7
omg = 1
omga = -omg
Omg = 0
dz = 0.01
hz =5
v = 1
#########################################################
deg = theta/2/m.pi*360
vxp=m.sin(theta)*v
vxn=m.sin(theta)*(-v)
vz=m.cos(theta)*v
print('deg',deg)
zwpi = False
######################################################
def hvv(vxp,vxn,mu):
    return mu*(1-(vz*vz+vxp*vxn))/2#2*mu*(E**3*g*(1-(vz*vz+vxp*vxn)))#/(2*m.pi)**3
def hvva(vxp,vxn,mu):
    return mu*(1-(vz*vz+vxp*vxn))*(-1)*ar/2
def fmu(z):
    mu= 10**4*math.exp(-0.3*z)
    return mu
def Lda(vxp,vxn,vz,mu):
    #2*mu*(E**3*g*(1-(vz*vz+vxp*vxn))*(1-ar))#/(2*m.pi)**3       #2*mu*(1-(vz*vz+vxp*vxn))*(E**2)*(1-ar)/((2*m.pi)**1)
    return mu*(1-ar)*(1-(vz*vz+vxp*vxn))/2
def lda0(mu):
    return Lmu*mu
#########################################################
def fim_eig(z,modk):
    LHSM = []
    #sys.exit()
    print(omg,'omg')
    print(omga,'omga')
    for i in range(4):
        A = []
        for j in range(4):
            A.append(0+0j)
        LHSM.append(A)
    for i in range(len(LHSM)):
        for j in range(len(LHSM[i])):
            if i == j:
                if i%4 ==0:
                    LHSM[i][j] = LHSM[i][j]+(-Omg+omg+vxp*modk)+Lda(vxp,vxn,vz,fmu(z))+lda0(fmu(z))
                if i%4 == 1:
                    LHSM[i][j] = LHSM[i][j]+(-Omg+omg+vxn*modk)+Lda(vxp,vxn,vz,fmu(z))+lda0(fmu(z))
                if i%4 == 2:
                    LHSM[i][j] = LHSM[i][j]+(-Omg+omga+vxp*modk)+Lda(vxp,vxn,vz,fmu(z))+lda0(fmu(z))
                if i%4 == 3:
                    LHSM[i][j] = LHSM[i][j]+(-Omg+omga+vxn*modk)+Lda(vxp,vxn,vz,fmu(z))+lda0(fmu(z))
        j =  i   #see test part
        if i%4 ==0:
            if j+1 < 4:
                LHSM[i][j+1] = LHSM[i][j+1]-hvv(vxp,vxn,fmu(z))
            if j+3 < 4:
                LHSM[i][j+3] = LHSM[i][j+3]-hvva(vxp,vxn,fmu(z))
        if i%4 == 2:
            if j-1 > -1:
                LHSM[i][j-1] = LHSM[i][j-1]-hvv(vxp,vxn,fmu(z))
            if j+1 < 4:
                LHSM[i][j+1] = LHSM[i][j+1]-hvva(vxp,vxn,fmu(z))
        if i%4 == 1:
            if j-1 > -1:
                LHSM[i][j-1] = LHSM[i][j-1]-hvv(vxp,vxn,fmu(z))
            if j+1 < 4:
                LHSM[i][j+1] = LHSM[i][j+1]-hvva(vxp,vxn,fmu(z))
        if i%4 == 3:
            if j-1 > -1:
                LHSM[i][j-1] = LHSM[i][j-1]-hvva(vxp,vxn,fmu(z))
            if j-3 > -1:    
                LHSM[i][j-3] = LHSM[i][j-3]-hvv(vxp,vxn,fmu(z))
    NCFM = LHSM
    NCM = np.array(NCFM)# / vz
    #print("zwpi",zwpi)
    print(NCM)
    l, vec= linalg.eig(NCM)
    Al = []
    Cii = []#"check if in"
    #iniQ = []
    for j in range(len(l)):
        Al.append(abs(((l[j]).imag)))
    Ml = np.max(Al)
    print('l',l)
    print(Al)
    """for j in range(len(Al)):
        if Al[j] == np.max(Al):
            if len(Cii) == 0:
                iniQ = vec[:,j]"""
    #sys.exit()
    #print("max eig val",Ml)
    #ML.append(Ml)
    del LHSM
    return Ml
print(fmu((np.log(100)-np.log(10000))/(-0.3)),'mu')
Lmu=0
omg=1
omga=-1
print(fim_eig((np.log(100)-np.log(10000))/(-0.3), 0))
omg=-1
omga=1
print(fim_eig((np.log(100)-np.log(10000))/(-0.3),0))    

deg 90.0
99.99999999999994 mu
1 omg
-1 omga
[[  31.+0.j -100.+0.j    0.+0.j   70.+0.j]
 [-100.+0.j   31.+0.j   70.+0.j    0.+0.j]
 [   0.+0.j -100.+0.j   29.+0.j   70.+0.j]
 [-100.+0.j    0.+0.j   70.+0.j   29.+0.j]]
l [64.89974874+0.j  7.51668523+0.j 22.48331477+0.j 25.10025126+0.j]
[0.0, 0.0, 0.0, 0.0]
0.0
-1 omg
1 omga
[[  29.+0.j -100.+0.j    0.+0.j   70.+0.j]
 [-100.+0.j   29.+0.j   70.+0.j    0.+0.j]
 [   0.+0.j -100.+0.j   31.+0.j   70.+0.j]
 [-100.+0.j    0.+0.j   70.+0.j   31.+0.j]]
l [-4.89974874+0.j 52.48331477+0.j 37.51668523+0.j 34.89974874+0.j]
[0.0, 0.0, 0.0, 0.0]
0.0


In [12]:
#Instability analysis new one to compare to the above
#!/hetghome/hetgsoft/anaconda3/bin/python3.9
%matplotlib inline
#sed -i 's/\r//g' RK45_wo_tur_kmin.py
#chmod 777 RK45_wo_tur_kmin.py
from cProfile import label
import matplotlib
#matplotlib.use('Agg')
import matplotlib.pyplot as plt
import math as m
import math
from scipy.optimize import fsolve
from scipy import linalg
import csv
import pandas as pd
import sys
import numpy as np
import os
from scipy.integrate import RK45
import time
import cmath 
acu = 1e-13
ts = 0.01
####################
#theta = m.pi/2
ar = 0.7
omg = 0.01
omga = -omg
v=1
mu=1
t1=600
t0=0
##############################################################
"""vxp=m.sin(theta)*v
vxn=m.sin(theta)*(-v)
vz=m.cos(theta)*v"""
#################################################
def absq(x):
    return cmath.sqrt(1-abs(x)**2)
def exa(a,b):
    return b-a
def tbif(t,y):
    dqdt=np.array([0+0j,0+0j,0+0j,0+0j])
    #dqdt=y
    #print('0-0-0-0-0-0-0-0-0-0',dqdt[0],dqdt[1],dqdt[2],dqdt[3])
    #print(type(dqdt[0]))
    #dqdt[0]=-omg*y[0]+0*0.5*mu*(absq(y[1])*y[0]-y[1]*absq(y[0]))-0*0.5*ar*mu*(absq(y[3])*y[0]-y[3]*absq(y[0]))
    dqdt[0]=-omg*y[0]+2/2*mu*(exa(y[1],y[0])-ar*exa(y[3],y[0]))+0/2*mu*(exa(y[0],y[0])-ar*exa(y[2],y[0]))
    dqdt[1]=-omg*y[1]+2/2*mu*(exa(y[0],y[1])-ar*exa(y[2],y[1]))+0/2*mu*(exa(y[1],y[1])-ar*exa(y[3],y[1]))
    dqdt[2]=-omga*y[2]+2/2*mu*(exa(y[1],y[2])-ar*exa(y[3],y[2]))+0/2*mu*(exa(y[0],y[2])-ar*exa(y[2],y[2]))
    dqdt[3]=-omga*y[3]+2/2*mu*(exa(y[0],y[3])-ar*exa(y[2],y[3]))+0/2*mu*(exa(y[1],y[3])-ar*exa(y[3],y[3]))
    #dqdt[0]+=-omg*y[0]
    #dqdt[1]+=-omg*y[1]
    #dqdt[2]+=-omga*y[2]
    #dqdt[3]+=-omga*y[3]
    #print('---------',dqdt[0],dqdt[1],dqdt[2],dqdt[3])
    dqdt=(-1j)*dqdt
    #print('>>>>>>>>>',dqdt[0],dqdt[1],dqdt[2],dqdt[3])
    return dqdt

for i in [0.01,-0.01]:
    omg=i
    Mat=[[-omg+2/2*mu-2/2*mu*ar-0/2*mu*ar, -2/2*mu, 0/2*mu*ar, 2/2*mu*ar],\
         [-2/2*mu, -omg+2/2*mu-2/2*mu*ar-0/2*mu*ar, 2/2*mu*ar, 0/2*mu*ar],\
         [-0/2*mu, -2/2*mu, -omga+2/2*mu-2/2*mu*ar+0/2*mu, 2/2*mu*ar],\
         [-2/2*mu, -0/2*mu, 2/2*mu*ar, -omga+2/2*mu-2/2*mu*ar+0/2*mu]]
    for j in Mat:
        print(j)
    #Mat = [[1,1],[1,-1]]
    l, vec= linalg.eig(Mat)
    print(omg,'omg')
    print(l,'eigen-val')
    #print('vec',vec)
    print('^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^')
    #print(np.transpose(vec))
    #print(np.matmul(np.transpose(vec),vec))
    """for j in vec:
        print(vec)"""
    print('sym',np.matmul(np.transpose(vec),[1,1,1,1]))
    print('asy',np.matmul(np.transpose(vec),[1,0.1,1,0.1]))
    print('>>>>>>>>>>>')
    continue
    TTTT=vec[0]
    for j in range(4):
        plt.scatter(np.real(vec[j]),np.imag(vec[j]),label=str(j)+'-th')
    plt.xlabel('Re')
    plt.ylabel('Im')
    plt.legend()
    plt.show()
    plt.close('all')
    for j in range(4):
        plt.plot(range(4),np.absolute(vec[j]))
        plt.title(str(j)+'-th '+r'$\omega$='+str(omg)+' Im[val]='+str(l[j].imag))
        plt.xticks([0,1,2,3])
        plt.xlabel('conponents')
        #plt.show()
        plt.close('all')
print('---------------------------------------------------')
VE = [1,1,1,1]
#print(tbif(0,VE))
print(np.matmul(Mat,VE))
#sys.exit()


[0.29000000000000004, -1.0, 0.0, 0.7]
[-1.0, 0.29000000000000004, 0.7, 0.0]
[0.0, -1.0, 0.31000000000000005, 0.7]
[-1.0, 0.0, 0.7, 0.31000000000000005]
0.01 omg
[-0.04899749+0.j  0.52483315+0.j  0.37516685+0.j  0.34899749+0.j] eigen-val
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
sym [-1.99917948e+00  9.99200722e-16 -2.22044605e-15  1.95960410e+00]
asy [-1.09954872 -0.89911111  0.89213966  1.07778225]
>>>>>>>>>>>
[0.31000000000000005, -1.0, 0.0, 0.7]
[-1.0, 0.31000000000000005, 0.7, 0.0]
[0.0, -1.0, 0.31000000000000005, 0.7]
[-1.0, 0.0, 0.7, 0.31000000000000005]
-0.01 omg
[0.01+0.j 0.61+0.j 0.31+0.j 0.31+0.j] eigen-val
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
sym [-2.00000000e+00  9.99200722e-16  1.39269426e+00  1.61976598e+00]
asy [-1.1        -0.9         1.39269426  0.38663975]
>>>>>>>>>>>
---------------------------------------------------
[0.01 0.01 0.01 0.01]
